# CRISPR Constrained model For CAS 13b

In [7]:
import pandas as pd
import numpy as np
import pgmpy
from sklearn.model_selection import train_test_split

df_raw = pd.read_csv('Data_high_dosage.csv')
# print(df_raw.head())
# print(df_raw.shape)

df_spacer_split = df_raw['spacer_seq'].apply(lambda x: pd.Series(list(x)))
df_spacer_split.columns = [f'SP{str(i+1).zfill(2)}' for i in range(df_spacer_split.shape[1])]
df = pd.concat([df_spacer_split, df_raw.drop(['spacer_seq', 'dosage (ng)'], axis=1)], axis=1)
df['mean_targeting_efficiency'] = df['mean_targeting_efficiency'].apply(lambda x: ((x+0.05) // 0.1) / 10)
df.drop("transcript", axis=1, inplace=True)
# print(df.head())

train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)

print(train_df.head())
print(test_df.head())




    SP01 SP02 SP03 SP04 SP05 SP06 SP07 SP08 SP09 SP10  ... SP22 SP23 SP24  \
120    G    C    U    C    G    G    A    G    G    A  ...    C    C    A   
10     G    G    U    C    U    U    C    U    U    C  ...    G    C    C   
73     C    U    G    C    G    U    C    U    C    C  ...    C    C    U   
159    U    G    A    U    U    G    G    G    G    U  ...    A    G    A   
156    G    G    U    U    G    A    U    U    G    G  ...    A    G    U   

    SP25 SP26 SP27 SP28 SP29 SP30 mean_targeting_efficiency  
120    U    C    G    U    C    U                       0.7  
10     G    U    C    G    G    A                       1.0  
73     C    G    C    C    A    U                       0.9  
159    U    C    U    U    A    A                       0.9  
156    A    G    A    U    C    U                       0.9  

[5 rows x 31 columns]
    SP01 SP02 SP03 SP04 SP05 SP06 SP07 SP08 SP09 SP10  ... SP22 SP23 SP24  \
24     G    G    U    G    A    U    G    U    C    C  ...    G  

In [8]:
# make the model
from pgmpy.models import BayesianNetwork

nodes = list(df.columns)
print(nodes)

edges = [("SP01", 'mean_targeting_efficiency'), ("SP02", 'mean_targeting_efficiency'), ("SP11", 'mean_targeting_efficiency'),("SP12", 'mean_targeting_efficiency'),("SP15", 'mean_targeting_efficiency'),("SP16", 'mean_targeting_efficiency'),("SP17", 'mean_targeting_efficiency'),
         ("SP03", 'mean_targeting_efficiency'), ("SP04", 'mean_targeting_efficiency')]
model = BayesianNetwork(edges)

leftover_nodes = [x for x in nodes if x not in model.nodes]
model.add_nodes_from(leftover_nodes)

# model.add_nodes_from(nodes)


# model.add_edges_from([(df.columns[-1], s) for s in df.columns[:-1]])
# for s in df.columns[:-1]:
    
    # model.add_edge(df.columns[-1], s)


model.to_graphviz().draw("const.png", prog="dot")

['SP01', 'SP02', 'SP03', 'SP04', 'SP05', 'SP06', 'SP07', 'SP08', 'SP09', 'SP10', 'SP11', 'SP12', 'SP13', 'SP14', 'SP15', 'SP16', 'SP17', 'SP18', 'SP19', 'SP20', 'SP21', 'SP22', 'SP23', 'SP24', 'SP25', 'SP26', 'SP27', 'SP28', 'SP29', 'SP30', 'mean_targeting_efficiency']


In [9]:
from pgmpy.estimators import HillClimbSearch, BicScore, K2Score
from pgmpy.models import BayesianNetwork
import time

fixed_edges = edges
nodes_to_learn = [node for node in df.columns if node not in ['mean_targeting_efficiency'] + [edge[0] for edge in fixed_edges]]

est = HillClimbSearch(train_df, use_cache=True)
# est = HillClimbSearch(test_df, use_cache=True)
# dag = est.estimate(start_dag=model, scoring_method="bdeuscore", fixed_edges=fixed_edges)
dag = est.estimate(start_dag=model, scoring_method="bdeuscore")
# dag = est.estimate(scoring_method="bdeuscore")

final_model = BayesianNetwork(dag)
final_model.to_graphviz().draw("final_model_testdata.png", prog="dot")

  0%|          | 0/1000000 [00:00<?, ?it/s]

In [10]:
from pgmpy.estimators import MaximumLikelihoodEstimator
final_model.fit(train_df, estimator=MaximumLikelihoodEstimator)

from pgmpy.readwrite import XMLBIFWriter
writer = XMLBIFWriter(final_model)
writer.write_xmlbif("final_model_testdata.xml")

cpds = final_model.get_cpds()
for cpd in cpds:
    print(cpd)


+---------+---------+-----+---------------------+
| SP25    | SP25(A) | ... | SP25(U)             |
+---------+---------+-----+---------------------+
| SP01(A) | 0.1875  | ... | 0.1956521739130435  |
+---------+---------+-----+---------------------+
| SP01(C) | 0.28125 | ... | 0.17391304347826086 |
+---------+---------+-----+---------------------+
| SP01(G) | 0.28125 | ... | 0.2826086956521739  |
+---------+---------+-----+---------------------+
| SP01(U) | 0.25    | ... | 0.34782608695652173 |
+---------+---------+-----+---------------------+
+---------+----------+
| SP02(A) | 0.172727 |
+---------+----------+
| SP02(C) | 0.222727 |
+---------+----------+
| SP02(G) | 0.359091 |
+---------+----------+
| SP02(U) | 0.245455 |
+---------+----------+
+---------+----------+
| SP03(A) | 0.172727 |
+---------+----------+
| SP03(C) | 0.281818 |
+---------+----------+
| SP03(G) | 0.286364 |
+---------+----------+
| SP03(U) | 0.259091 |
+---------+----------+
+---------+---------------------+---

In [11]:
test_pred = final_model.predict(test_df.drop('mean_targeting_efficiency', axis=1))
print(test_pred)

  0%|          | 0/25 [00:00<?, ?it/s]

    mean_targeting_efficiency
0                        -0.1
1                        -0.1
2                        -0.1
3                        -0.1
4                        -0.1
5                        -0.1
6                        -0.1
7                        -0.1
8                        -0.1
9                        -0.1
10                       -0.1
11                       -0.1
12                       -0.1
13                       -0.1
14                       -0.1
15                       -0.1
16                       -0.1
17                       -0.1
18                       -0.1
19                       -0.1
20                       -0.1
21                       -0.1
22                       -0.1
23                       -0.1
24                       -0.1


In [12]:
errors = test_df['mean_targeting_efficiency'] - test_pred['mean_targeting_efficiency']
print(f"Errors: {errors}")
print(f"MSE Loss: {np.mean(errors**2)}")

Errors: 0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
5      NaN
6      1.1
7      NaN
8      NaN
9      1.1
10     NaN
11     NaN
12     NaN
13     NaN
14     NaN
15     0.7
16     NaN
17     NaN
18     NaN
19     0.5
20     NaN
21     NaN
22     NaN
23     NaN
24     1.1
30     NaN
33     NaN
45     NaN
66     NaN
82     NaN
101    NaN
109    NaN
117    NaN
124    NaN
142    NaN
146    NaN
153    NaN
172    NaN
177    NaN
190    NaN
193    NaN
199    NaN
206    NaN
211    NaN
212    NaN
Name: mean_targeting_efficiency, dtype: float64
MSE Loss: 0.8740000000000002


In [13]:
from pgmpy.estimators import HillClimbSearch, BicScore, K2Score
from pgmpy.models import BayesianNetwork
import time

def learn_constrained(data, fixed_edges):
    starttime = time.time()
    hc = HillClimbSearch(data)
    learned_model = hc.estimate(scoring_method=K2Score(data), fixed_edges=fixed_edges)
    # learned_model = hc.estimate(scoring_method=BicScore(data), fixed_edges=fixed_edges)
    final_edges = fixed_edges + list(learned_model.edges)
    final_model = BayesianNetwork(final_edges)
    return final_model

In [14]:
fixed_edges = edges
nodes_to_learn = [node for node in df.columns if node not in ['mean_targeting_efficiency'] + [edge[0] for edge in fixed_edges]]

In [15]:
fixed_edges, nodes_to_learn

([('SP01', 'mean_targeting_efficiency'),
  ('SP02', 'mean_targeting_efficiency'),
  ('SP11', 'mean_targeting_efficiency'),
  ('SP12', 'mean_targeting_efficiency'),
  ('SP15', 'mean_targeting_efficiency'),
  ('SP16', 'mean_targeting_efficiency'),
  ('SP17', 'mean_targeting_efficiency'),
  ('SP03', 'mean_targeting_efficiency'),
  ('SP04', 'mean_targeting_efficiency')],
 ['SP05',
  'SP06',
  'SP07',
  'SP08',
  'SP09',
  'SP10',
  'SP13',
  'SP14',
  'SP18',
  'SP19',
  'SP20',
  'SP21',
  'SP22',
  'SP23',
  'SP24',
  'SP25',
  'SP26',
  'SP27',
  'SP28',
  'SP29',
  'SP30'])

In [16]:
# learned_bn = learn_constrained(test_df, fixed_edges)
learned_bn = learn_constrained(train_df, fixed_edges)

learned_bn.to_graphviz().draw("learned_train_1.png", prog="dot")

  0%|          | 0/1000000 [00:00<?, ?it/s]

/home/nightwing/.local/lib/python3.12/site-packages/pgmpy/estimators/StructureScore.py:136: RuntimeWarning: overflow encountered in scalar multiply
  (num_parents_states - counts.shape[1]) * var_cardinality * gammaln(1)
/home/nightwing/.local/lib/python3.12/site-packages/pgmpy/estimators/StructureScore.py:136: RuntimeWarning: overflow encountered in scalar multiply
  (num_parents_states - counts.shape[1]) * var_cardinality * gammaln(1)
/home/nightwing/.local/lib/python3.12/site-packages/pgmpy/estimators/StructureScore.py:136: RuntimeWarning: overflow encountered in scalar multiply
  (num_parents_states - counts.shape[1]) * var_cardinality * gammaln(1)
/home/nightwing/.local/lib/python3.12/site-packages/pgmpy/estimators/StructureScore.py:136: RuntimeWarning: overflow encountered in scalar multiply
  (num_parents_states - counts.shape[1]) * var_cardinality * gammaln(1)
/home/nightwing/.local/lib/python3.12/site-packages/pgmpy/estimators/StructureScore.py:136: RuntimeWarning: overflow enc

In [126]:
learned_bn.to_graphviz().draw("learned.png", prog="dot")

In [19]:
writer = XMLBIFWriter(learned_bn)
writer.write_xmlbif("learned_train_1.xml")

cpds = learned_bn.get_cpds()
for cpd in cpds:
    print(cpd)

In [22]:
test_pred_learned = learned_bn.predict(test_df.drop('mean_targeting_efficiency', axis=1))
print(test_pred_learned)

ValueError: No CPD associated with SP01